In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
train_loc = train.iloc[:,30:]

#####################
a = train_data.columns.values.tolist()
m =[0,1,3,4,5,6,7,8,9,11,12,13,21,25]
order=[]
for i in m:
    order.extend([a[i]])
train_data = train_data[order]
##############

x_test = test.iloc[:,:30].applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
a = x_test.columns.values.tolist()
m =[0,1,3,4,5,6,7,8,9,11,12,13,21,25]
order=[]
for i in m:
    order.extend([a[i]])
x_test = x_test[order]
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))
#x_test = true_test
'''
x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))'''
###################
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]

feat_dim = train_data.shape[0]

def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
        #x_valid = valid_features.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_select_14.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_select_14.t7')
    if use_gpu:
        x_test = x_test.cuda()
    a = Tensor.cpu(net(x_test)).detach().numpy()
    #a = net(x_test).detach().numpy()
    a = np.argmax(a,axis=1)
    with open('./ans_select_14.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')

# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:39.123223742723155
epoch:1
error:39.0469558538163
epoch:2
error:38.963979624838295
epoch:3
error:38.88264826568564
epoch:4
error:38.79370654107374
epoch:5
error:38.70044166397154
epoch:6
error:38.59879831015524
epoch:7
error:38.48693958198577
epoch:8
error:38.36946151358344
epoch:9
error:38.24533574547219
epoch:10
error:38.11453448415265
epoch:11
error:37.97416215232859
epoch:12
error:37.83190238923027
epoch:13
error:37.678527753072444
epoch:14
error:37.51479877506468
epoch:15
error:37.34534938955369
epoch:16
error:37.172142828266495
epoch:17
error:36.99143960219922
epoch:18
error:36.79384045520698
epoch:19
error:36.59436903703105
epoch:20
error:36.389505679980594
epoch:21
error:36.17607181840233
epoch:22
error:35.97247432891332
epoch:23
error:35.7431577457956
epoch:24
error:35.51154946232212
epoch:25
error:35.27771769890039
epoch:26
error:35.04361305384864
epoch:27
error:34.77817250161708
epoch:28
error:34.52691926746442
epoch:29
error:34.273998423350584
epoch:30
error:3

epoch:243
error:4.423514816461837
epoch:244
error:4.406017545278137
epoch:245
error:4.449480893232535
epoch:246
error:4.486516362791074
epoch:247
error:4.424026469518111
epoch:248
error:4.371605403056274
epoch:249
error:4.471450391130336
epoch:250
error:4.369518995593467
epoch:251
error:4.370882772073092
epoch:252
error:4.391247195383247
epoch:253
error:4.328030249434024
epoch:254
error:4.29976520809751
epoch:255
error:4.339986507519405
epoch:256
error:4.2522560109759056
epoch:257
error:4.267814369744502
epoch:258
error:4.307972186287193
epoch:259
error:4.3430425291073735
epoch:260
error:4.71533076790912
epoch:261
error:4.6149078013826
epoch:262
error:4.4787158645294305
epoch:263
error:4.434952157280886
epoch:264
error:4.5023084396224125
epoch:265
error:4.447196772922057
epoch:266
error:4.416048156027652
epoch:267
error:4.4446454560155235
epoch:268
error:4.41223318240621
epoch:269
error:4.3176102138583445
epoch:270
error:4.34183856524903
epoch:271
error:4.273704696596054
epoch:272
erro

epoch:482
error:3.263932123221216
epoch:483
error:3.195373140362225
epoch:484
error:3.1879377476148125
epoch:485
error:3.2076035434184993
epoch:486
error:3.1751271552595406
epoch:487
error:3.422758391110123
epoch:488
error:3.352303449425938
epoch:489
error:3.2641974248059507
epoch:490
error:3.278953877951164
epoch:491
error:3.2557431476390684
epoch:492
error:3.1988833329964423
epoch:493
error:3.201362639472833
epoch:494
error:3.1663977856161063
epoch:495
error:3.1963042225905562
epoch:496
error:3.1702282351633246
epoch:497
error:3.134445052251779
epoch:498
error:3.167191163688551
epoch:499
error:3.1557797213575354
epoch:500
error:3.281245262471701
epoch:501
error:3.234224346600097
epoch:502
error:3.230668989529431
epoch:503
error:3.1857641696313066
epoch:504
error:3.466206894809185
epoch:505
error:3.467916195019405
epoch:506
error:3.390754492440168
epoch:507
error:3.255720407503234
epoch:508
error:3.217940830166559
epoch:509
error:3.2832527111295278
epoch:510
error:3.141157182345569
ep

epoch:720
error:2.8481644288688552
epoch:721
error:2.8559128143192107
epoch:722
error:2.844364615337969
epoch:723
error:2.970466880255498
epoch:724
error:2.8901967274417855
epoch:725
error:2.862646421207956
epoch:726
error:2.870166457794308
epoch:727
error:2.932565074688713
epoch:728
error:2.884136165406695
epoch:729
error:2.864688611739974
epoch:730
error:2.881760137047219
epoch:731
error:2.86885605746685
epoch:732
error:2.888851269404916
epoch:733
error:2.8676947313631955
epoch:734
error:2.9390327483222833
epoch:735
error:2.97257413284282
epoch:736
error:2.8983427494340233
epoch:737
error:2.8764800038405562
epoch:738
error:2.9166289769970892
epoch:739
error:2.8818709952094115
epoch:740
error:2.9094206697728007
epoch:741
error:2.9194642297663327
epoch:742
error:2.98282740742238
epoch:743
error:3.165511867824224
epoch:744
error:3.1062008560397802
epoch:745
error:2.9396262027005173
epoch:746
error:2.901842203670763
epoch:747
error:2.9169009111214423
epoch:748
error:2.902555991267788
epo

epoch:958
error:3.019353118935964
epoch:959
error:2.864649448172704
epoch:960
error:2.802925139472833
epoch:961
error:2.7943900084896507
epoch:962
error:2.7976292145051747
epoch:963
error:2.732460091061611
epoch:964
error:2.7992083906047864
epoch:965
error:2.890918410919308
epoch:966
error:2.78759702457956
epoch:967
error:2.758206662354463
epoch:968
error:2.8634754886602525
epoch:969
error:2.8246643934953104
epoch:970
error:2.786711422622898
epoch:971
error:2.761889301018758
epoch:972
error:2.7355739104948253
epoch:973
error:2.7194476800008087
epoch:974
error:2.799865327862225
epoch:975
error:2.8013652293216365
epoch:976
error:2.777726542286546
epoch:977
error:2.7261373857939843
epoch:978
error:2.779471847711837
epoch:979
error:2.723762620775388
epoch:980
error:2.7088018222428847
epoch:981
error:2.6932753628315007
epoch:982
error:2.7260855888179174
epoch:983
error:2.7729267944493854
epoch:984
error:2.73201413173108
epoch:985
error:2.6828603806193403
epoch:986
error:2.73828061832956
epo

epoch:1190
error:2.752812196798189
epoch:1191
error:2.7396371305991267
epoch:1192
error:2.7781225996523284
epoch:1193
error:2.7170691249595733
epoch:1194
error:2.768128309953105
epoch:1195
error:2.7653097964505173
epoch:1196
error:2.7324013457107053
epoch:1197
error:2.7099110355352525
epoch:1198
error:2.7694668196151357
epoch:1199
error:3.119645013846216
epoch:1200
error:2.9060522871523284
epoch:1201
error:2.86275191017141
epoch:1202
error:2.8014034453832473
epoch:1203
error:2.758719894586837
epoch:1204
error:2.7683178110850584
epoch:1205
error:2.792052827862225
epoch:1206
error:2.808337923471863
epoch:1207
error:2.7739955808336028
epoch:1208
error:2.7252044085543337
epoch:1209
error:2.798266885814198
epoch:1210
error:2.7788272280279753
epoch:1211
error:2.755363197869502
epoch:1212
error:2.73628706642141
epoch:1213
error:2.7652365226794955
epoch:1214
error:2.7770525499272316
epoch:1215
error:2.741312320605595
epoch:1216
error:2.756091198051423
epoch:1217
error:2.7342714060478657
epoch:

epoch:1422
error:2.7263401520051747
epoch:1423
error:2.6493379462119986
epoch:1424
error:2.672530673916559
epoch:1425
error:2.637082118420521
epoch:1426
error:2.679643598904431
epoch:1427
error:2.690783107111093
epoch:1428
error:2.6715825365863517
epoch:1429
error:2.7012757847873545
epoch:1430
error:2.6888644081500646
epoch:1431
error:2.6872005882115135
epoch:1432
error:2.6958816350663
epoch:1433
error:2.6565089848803365
epoch:1434
error:2.8670235815208605
epoch:1435
error:2.7668403339262615
epoch:1436
error:2.6633518707551747
epoch:1437
error:2.6966247953387774
epoch:1438
error:2.6874662056314684
epoch:1439
error:2.7165546293863194
epoch:1440
error:2.6878733172299483
epoch:1441
error:2.686290982778137
epoch:1442
error:2.7187705292892947
epoch:1443
error:2.6694316987386806
epoch:1444
error:2.637650148063551
epoch:1445
error:2.619428824638179
epoch:1446
error:2.6716532836756146
epoch:1447
error:2.6780615802878396
epoch:1448
error:2.6542059144566625
epoch:1449
error:2.6456281457187902
ep

In [ ]:
y_valid = valid_labels#.cuda()
x_valid = valid_features#.cuda()



net = torch.load('model_NN_select_14.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)